In [170]:
import pycrfsuite
import codecs
import pandas as pd
import numpy as np
import os
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
from itertools import chain
from pyltp import Postagger
from pyltp import Segmentor

In [229]:
# read sentance, read label, segment and pos tag the sentence
def process1sent(labelsfilename,sentfilename,labelsChn2Eng):
    postagger = Postagger() # 初始化实例
    postagger.load("/home/user/Python_notebooks/chinese_nlp/ltp-cws/ltp_data_v3.4.0/pos.model")  # 加载模型
    segmentor = Segmentor()
    segmentor.load("/home/user/Python_notebooks/chinese_nlp/ltp-cws/ltp_data_v3.4.0/cws.model")

    #segment and pos tag
    with open(sentfilename,'r') as f:
        sent = f.read()
    sent = sent.replace('\n','')
    words = list(segmentor.segment(sent))
    #print(" ".join(words))
    segmentor.release()

    postags = list(postagger.postag(words))  # 词性标注
    #print(' '.join(postags))
    postagger.release()  # 释放模型
    
    # assign pos tag to each character
    chars = []
    postags_c = []
    for word,postag in zip(words,postags):
        for char in word:
            chars.append(char)
            postags_c.append(postag)
    if (len(chars) != (len(sent))):
        print(sent)
        print(''.join(chars))
        return(None)
    
    # associate words, postags and labels
    sent_list = []
    if (os.path.getsize(labelsfilename) == 0): # no labels in the sentence
        for char,postag in zip(chars,postags_c):
            sent_list.append((char,postag,'O'))
    else:
        #read labels
        labels = pd.read_csv(labelsfilename,sep='\t',header=None)
        labels_c = ['O'] * len(chars)
        for row in range(0,labels.shape[0]):
            s_idx = labels.iloc[row][1]
            e_idx = labels.iloc[row][2]
            label_chn = labels.iloc[row][3]
            label_eng = labelsChn2Eng[label_chn]
            if s_idx == e_idx:
                labels_c[s_idx] = 'S_' + label_eng
            else:
                labels_c[s_idx] = 'B_' + label_eng
                labels_c[s_idx+1:e_idx] = ['I_' + label_eng] * (e_idx-s_idx-1)
                labels_c[e_idx] = 'E_' + label_eng
                
        for char,postag,label in zip(chars,postags_c,labels_c):
            sent_list.append((char,postag,label))
    return sent_list

In [196]:
labelsChn2Eng = {'检查和检验':'check','疾病和诊断':'disease&diagnose','症状和体征':'symptom','身体部位':'body'}
filepath = '/home/user/Python_notebooks/chinese_nlp/ner-data/CCKS2017/CCKS2017_dataset/case_of_illness/data/trainingset 1-100/一般项目/'

train_sent = []
for i in range(1,101):   
    labelsfilename = filepath + '一般项目-'+str(i)+'.txt'
    sentfilename = filepath + '一般项目-'+str(i)+'.txtoriginal.txt'
    train_sent.append(process1sent(labelsfilename,sentfilename,labelsChn2Eng))
train_sent[0]

ArgumentError: Python argument types in
    Segmentor.segment(Segmentor, list)
did not match C++ signature:
    segment(Segmentor {lvalue}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)

In [230]:
labelsChn2Eng = {'检查和检验':'check','疾病和诊断':'disease&diagnose','症状和体征':'symptom','身体部位':'body','治疗':'treatment'}
filepath = '/home/user/Python_notebooks/chinese_nlp/ner-data/CCKS2017/CCKS2017_dataset/case_of_illness/data/trainingset 1-100/出院情况/'

train_sent = []
for i in range(1,101):   
    labelsfilename = filepath + '出院情况-'+str(i)+'.txt'
    sentfilename = filepath + '出院情况-'+str(i)+'.txtoriginal.txt'
    print(i)
    train_sent.append(process1sent(labelsfilename,sentfilename,labelsChn2Eng))
train_sent[0]

1
2
一般情况可，心肺未见异常。腹平坦，未见胃肠型及蠕动波。上腹部压痛阳性，无反跳痛及肌紧张。肝脾肋下未触及，全腹未触及 异常包块。腹叩移动性浊音阴性，肠鸣音正常，未闻及高调肠鸣音及气过水声。
一般情况可，心肺未见异常。腹平坦，未见胃肠型及蠕动波。上腹部压痛阳性，无反跳痛及肌紧张。肝脾肋下未触及，全腹未触及异常包块。腹叩移动性浊音阴性，肠鸣音正常，未闻及高调肠鸣音及气过水声。
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
患者神志清，精神尚好，无明显头痛、头晕，无恶心，无抽搐，无发热，二便正常。查体：头部创口 敷料包扎固定好，无渗出，双眼瞳孔对光反射灵敏，心肺腹听诊未见明显异常，四肢活动无异常，病理征未引出。患者病情好转，要求今日出院。
患者神志清，精神尚好，无明显头痛、头晕，无恶心，无抽搐，无发热，二便正常。查体：头部创口敷料包扎固定好，无渗出，双眼瞳孔对光反射灵敏，心肺腹听诊未见明显异常，四肢活动无异常，病理征未引出。患者病情好转，要求今日出院。
21
22
23
24
25
26
27
28
29
30
31
32
查体：T：36.5℃、P：78次/分、R：18次/分、BP130/80 mmHg，体型偏瘦，神志清楚，语言流利，表情自然，步入病室，自动体位，查体合作。心肺腹阴性，腹平坦，肝脾肋下未及，全腹未触及异常包块，无压痛反跳痛及肌紧张。腹叩移动性浊音阴性；肠鸣音正常，未闻及高调肠鸣音及气过水声。脊柱生理弯曲正常，无侧弯及后突畸形，各椎体棘突无压痛及叩击痛。脊肋角无隆起，双肾区叩击痛阴性。肛门外生殖无畸形。四肢无畸形，各关节活动正常；肌力肌张力正常；肱二、三肌腱反射，跟膝腱反射正常存在，巴氏等病理反射征未引出。右腹股沟部无包块，局部切口愈合良好，无红肿渗出。双侧睾丸无缺如，大小正常。窗体底  窗体底端
查体：T：36.5℃、P：78次/分、R：18次/分、BP130/80mmHg，体型偏瘦，神志清楚，语言流利，表情自然，步入病室，自动体位，查体合作。心肺腹阴性，腹平坦，肝脾肋下未及，全腹未触及异常包块，无压痛反跳痛及肌紧张。腹叩移动性浊音阴性；肠鸣音正常，未闻及高调肠鸣音及气过水声。脊柱生理弯曲正常，无侧弯及后突畸形，各椎体棘突无压痛及叩击痛。脊肋角无隆起，双肾区叩击痛阴性。肛门外生殖无畸形。四肢无畸形，各关

[('患', 'n', 'O'),
 ('者', 'n', 'O'),
 ('精', 'n', 'O'),
 ('神', 'n', 'O'),
 ('状', 'n', 'O'),
 ('况', 'n', 'O'),
 ('好', 'a', 'O'),
 ('，', 'wp', 'O'),
 ('无', 'v', 'O'),
 ('发', 'v', 'B_symptom'),
 ('热', 'v', 'E_symptom'),
 ('，', 'wp', 'O'),
 ('诉', 'v', 'O'),
 ('右', 'v', 'B_body'),
 ('髋', 'n', 'I_body'),
 ('部', 'n', 'E_body'),
 ('疼', 'a', 'B_symptom'),
 ('痛', 'a', 'E_symptom'),
 ('，', 'wp', 'O'),
 ('饮', 'n', 'O'),
 ('食', 'n', 'O'),
 ('差', 'a', 'O'),
 ('，', 'wp', 'O'),
 ('二', 'm', 'B_body'),
 ('便', 'd', 'E_body'),
 ('正', 'a', 'O'),
 ('常', 'a', 'O'),
 ('，', 'wp', 'O'),
 ('查', 'v', 'O'),
 ('体', 'v', 'O'),
 ('：', 'wp', 'O'),
 ('神', 'i', 'O'),
 ('清', 'i', 'O'),
 ('，', 'wp', 'O'),
 ('各', 'r', 'O'),
 ('项', 'r', 'O'),
 ('生', 'n', 'O'),
 ('命', 'n', 'O'),
 ('体', 'n', 'O'),
 ('征', 'n', 'O'),
 ('平', 'a', 'O'),
 ('稳', 'a', 'O'),
 ('，', 'wp', 'O'),
 ('心', 'n', 'B_body'),
 ('肺', 'n', 'I_body'),
 ('腹', 'n', 'E_body'),
 ('查', 'v', 'O'),
 ('体', 'v', 'O'),
 ('未', 'd', 'O'),
 ('见', 'v', 'O'),
 ('异', 'a', 'O'),
 (

In [240]:
train_sent2 = [item for item in train_sent if item is not None]
len(train_sent2)

93

In [154]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word=' + word1,
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word=' + word1,
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [155]:
sent2features(train_sent[0])[21]

['bias',
 'word=右',
 'word.isdigit=False',
 'postag=nd',
 'postag[:2]=nd',
 '-1:word=因',
 '-1:word.isdigit=False',
 '-1:postag=n',
 '-1:postag[:2]=n',
 '+1:word=髋',
 '+1:word.isdigit=False',
 '+1:postag=nd',
 '+1:postag[:2]=nd']

In [156]:
%%time
X = [sent2features(s) for s in train_sent]
y = [sent2labels(s) for s in train_sent]

X_train = X[:80]
y_train = y[:80]
X_test = X[80:]
y_test = y[80:]

CPU times: user 52 ms, sys: 12 ms, total: 64 ms
Wall time: 64 ms


In [160]:
# -*- coding: utf-8 -*-
# train the model
trainer = pycrfsuite.Trainer(verbose=False)
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [161]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [162]:
%%time
trainer.train('ccks.crfsuite')

CPU times: user 544 ms, sys: 0 ns, total: 544 ms
Wall time: 542 ms


In [165]:
tagger = pycrfsuite.Tagger()
tagger.open('ccks.crfsuite')

example_sent = train_sent[83]
print(' '.join(sent2tokens(example_sent)), end='\n\n')

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

沟 可 复 性 肿 物 1 年 于 2 0 1 6 - - 0 8 - - 1 1 0 8 : 4 0 收 入 院

Predicted: O O O O B_symptom E_symptom O O O O O O O O O O O O O O O O O O O O O O O
Correct:   O O O O B_symptom E_symptom O O O O O O O O O O O O O O O O O O O O O O O


### Evaluate the model

In [183]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
    
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [182]:
print(bio_classification_report(y_test, y_pred))

{'E_symptom', 'B_body', 'E_body', 'S_symptom', 'O', 'I_body', 'B_symptom'}
             precision    recall  f1-score   support

     B_body       0.80      0.89      0.84         9
  B_symptom       1.00      0.80      0.89        25
     E_body       0.90      1.00      0.95         9
  E_symptom       1.00      0.80      0.89        25
     I_body       1.00      0.80      0.89         5
  S_symptom       0.00      0.00      0.00         3

avg / total       0.93      0.80      0.86        76



/home/user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [172]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]
y_pred

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 15.9 ms


In [145]:
chinese_labels = []
for sent in train_sent:
    for char,postag,label in sent:
        chinese_labels.append(label)
chinese_labels = set(chinese_labels)
chinese_labels

{'B_检查和检验',
 'B_疾病和诊断',
 'B_症状和体征',
 'B_身体部位',
 'E_检查和检验',
 'E_疾病和诊断',
 'E_症状和体征',
 'E_身体部位',
 'I_疾病和诊断',
 'I_症状和体征',
 'I_身体部位',
 'O',
 'S_症状和体征',
 'S_身体部位'}

In [224]:
test = '患者全般及精神状况好，饮食及睡眠好，二便正常。诉牙齿松动。查体：前臂外伤结痂，无红肿。口腔内黏膜愈合，牙齿松动。'
test[53:55]

'松动'

In [225]:
[c for c in test]

['患',
 '者',
 '全',
 '般',
 '及',
 '精',
 '神',
 '状',
 '况',
 '好',
 '，',
 '饮',
 '食',
 '及',
 '睡',
 '眠',
 '好',
 '，',
 '二',
 '便',
 '正',
 '常',
 '。',
 '诉',
 '牙',
 '齿',
 '松',
 '动',
 '。',
 '查',
 '体',
 '：',
 '前',
 '臂',
 '外',
 '伤',
 '结',
 '痂',
 '，',
 '无',
 '红',
 '肿',
 '。',
 '口',
 '腔',
 '内',
 '黏',
 '膜',
 '愈',
 '合',
 '，',
 '牙',
 '齿',
 '松',
 '动',
 '。']